In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt, Hannanum, Kkma
import re
import collections
from tqdm import tqdm

# 1. 데이터 불러오기

In [2]:
path = 'D:/Summer 2021/교류학술제/data_crawling/'
assembly_r = pd.read_excel(path + "국회.xlsx")
blue_r = pd.read_excel(path + '청와대(2).xlsx') # 결측치 채워진 버전

# 2. 불필요한 열 삭제

In [3]:
blue = blue_r.drop(['index', 'end', 'state'], axis = 1).copy()
assembly = assembly_r.drop(['state', 'end'], axis = 1).copy()

# 3. 불필요한 문자열 제거

In [4]:
blue.text = blue.text.str.rstrip('[본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]')

In [5]:
def remove_signs_from_text(df):
    for i in range(len(df)):
        df.iloc[i].text = re.sub('[^가-힝\\s]', '', df.iloc[i].text)
    return df

In [6]:
assembly = remove_signs_from_text(assembly)
blue = remove_signs_from_text(blue)

In [7]:
def remove_comma_from_count(df):
    df['count'] = df['count'].str.replace(',', '')
    return df

In [8]:
assembly = remove_comma_from_count(assembly)
blue = remove_comma_from_count(blue)

# 4. 행 삭제

아래 토큰화 과정을 진행할 때, 아래의 행들의 토큰화에서 지속적으로 문제가 발생함. 
확인해본 결과, 문자열에 문제가 있는 것으로 판단되어 불가피하게 이 행들을 삭제함. 

In [9]:
index_to_delete = [1800, 6065, 6411, 9427, 10267, 10574, 12386, 15304, 15108]
for i in index_to_delete:
    print(blue.iloc[i])

title                                 외국인을위한 운전 면허 시험
count                                             121
categ                                              기타
start                                      2020-03-02
text                                                 
Name: 1800, dtype: object
title                                      제친구의 억울함을 들어주세요
count                                                 1475
categ                                               인권/성평등
start                                           2020-07-03
text                                                   ...
Name: 6065, dtype: object
title                            청소년 범죄, 그에 맞는 처벌을 내려주세요 !
count                                                  358
categ                                                   기타
start                                           2020-07-14
text                                                   ...
Name: 6411, dtype: object
title                          Spreading Malicious Rumours
cou

In [10]:
blue = blue.drop(index_to_delete); blue

,title,count,categ,start,text
0,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,267,인권/성평등,2020-01-02,본인은 년 월 경 서울지방병무청 제검사장 탈의실에서 믿을 수 없는 것을 발견하였다 ...
1,주식시장 활성화 및 소액(개미)투자자 보호,271,경제민주화,2020-01-02,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...
2,교정기관의 민낮,198,행정,2020-01-02,억울한 일로 국민청원을 신청합니다 저는 구치소 교도관입니다 이주 전 ...
3,미세먼지 저감 대책,170,안전/환경,2020-01-02,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다 우리 일상에서 가장 ...
4,악질세입자 방지를 위한 세입자보호법을 재정해주세요.,2127,교통/건축/국토,2020-01-02,저는 우선 아이셋의 부모입니다 식구가 많은편이고 아이들이 성장함에 따라 집이 좁아...
...,...,...,...,...,...
15310,"충북 오창 사는 33살 동생이 신경외과에서 주사 한대로 심정지, 사망하였습니다",16307,보건복지,2021-07-05,충북 오창 에 사는 제 동생은 디스크와 목디스크로 집 가까이 신경외과 신경차단술 ...
15311,오직 코로나 백신을 위해 9평 응시 신청을 한 분들의 응시 권한을 재수생과 검정고시...,387,보건복지,2021-07-05,안녕하세요 저는 고등학교에 재학 중인 한 고등학생입니다 여러 매체를 통해 이 이...
15312,보이스피싱 피해자의 억울함을 호소합니다. 진화하고 있는 신종 보이스피싱에 대한 수사...,352,안전/환경,2021-07-05,저는 대한민국의 대의 사회 초년생이자 취업을 준비하고 있는 취준생 입니다 이 ...
15313,사기죄? 왜 아닙니까?,213,기타,2021-07-05,안녕하세요 저는 사기죄가 무엇인지 알고싶습니다 저희 엄마께서 년전에 를 안고 세입...


# 5. 토큰화

In [11]:
def tokenize(df):
    text = df['text']
    kkm = Kkma()
    clean_words = pd.Series(0, index = df.index)
    for i in tqdm(range(len(text))):
        nouns = kkm.morphs(text.iloc[i])                 # 토큰화
        words = [w for w in nouns if len(w) >= 2]  # 분석을 위해 두글자 이상인 단어만 사용
        clean_words.iloc[i] = words
    clean_words = clean_words.astype(str)
    return clean_words

In [49]:
assembly_words = tokenize(assembly)
assembly_words

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [04:45<00:00,  1.30s/it]


0      ['최근', '국가', '보안법', '폐지', '청원', '올라오', '으나', '...
1      ['동의', '구현', '대하', '제재', '가하', '도록', '규정', '만들...
2      ['사회권', '위원회', '에서', '국가', '인권', '위원회', '에서', ...
3      ['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연...
4      ['하수도', '의하', '특정', '공산품', '주방', '오물', '분쇄', '...
                             ...                        
215    ['최근', '한국', '심리', '학회', '상담', '심리', '학회', '임상...
216    ['출판계', '플랫폼', '막아주', '세요', '한국', '출판계', '미래',...
217    ['일요일', '공휴일', '대하', '대체', '휴일', '일괄적', '으로', ...
218    ['순기능', '제대로', '못하', '역효과', '일으키', '셧다', '운제',...
219    ['세계', '민주주의', '여러', '나라', '가운데', '배심', '원제', ...
Length: 220, dtype: object

In [12]:
blue_words = tokenize(blue)

100%|██████████████████████████████████████████████████████████████████████████| 15306/15306 [6:25:55<00:00,  1.51s/it]


In [13]:
blue.text = blue_words
blue.to_excel("D:\Summer 2021\교류학술제\data_cleaning\청와대(토큰화).xlsx", index = False)

# 6. 불용어 제거

In [14]:
with open('D:/Summer 2021/교류학술제/data_cleaning/stopwords.txt', 'r', encoding = 'utf-8') as f:
    content = f.readlines()
stopwords = list(set([x.rstrip(', \n') for x in content]))

In [15]:
def remove_stopwords(series, stopwords):
    for word in stopwords:
        stopword = "'" + word + "', "
        series = series.str.replace(stopword, '')
        stopword = "'" + word + "'"
        series = series.str.replace(stopword, '')
    return series

In [15]:
assembly_clean_words = remove_stopwords(assembly_words, stopwords)

In [16]:
# 토큰화 과정이 6시간 가량 소요되기 때문에, 이를 생략하기 위해 토큰화가 완료된 데이터를 불러온다. 
blue_2 = pd.read_excel('D:/Summer 2021/교류학술제/data_cleaning/청와대(토큰화).xlsx' )
blue_words = blue_2['text']

In [17]:
blue_clean_words = remove_stopwords(blue_words, stopwords)
print(blue_words)
print(blue_clean_words)

0        ['본인', '서울', '지방', '병무청', '검사장', '탈의실', '에서', ...
1        ['우리', '나라', '코스', '미국', '애플', '보다', '다는', '정도...
2        ['억울', '국민', '청원', '신청', 'ㅂ니다', '구치소', '교도관', ...
3        ['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '습...
4        ['우선', '아이', '부모', 'ㅂ니다', '식구', '아이', '성장', '따...
                               ...                        
15301    ['충북', '동생', '디스크', '디스크', '가까이', '신경외과', '신경'...
15302    ['안녕', '세요', '고등학교', '재학', '중인', '고등학생', 'ㅂ니다'...
15303    ['대한민국', '대의', '사회', '초년', '이자', '취업', '준비', '...
15304    ['안녕', '세요', '사기죄', '무엇', 'ㄴ지', '습니다', '저희', '...
15305    ['대통령', '간곡히', '제언', '드리', 'ㅂ니다', '코로나', '이상',...
Name: text, Length: 15306, dtype: object
0        ['서울', '지방', '병무청', '검사장', '탈의실', '발견', '탈의실',...
1        ['나라', '코스', '미국', '애플', '한국', '주식', '시장', '평가...
2        ['억울', '국민', '신청', '구치소', '교도관', '이주', '구치소', ...
3        ['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '일...
4        ['아이', '부모', '식구', '아이', '성장', '나름', '단꿈', '가지...
               

# 7. 데이터 저장

In [34]:
assembly.text = assembly_clean_words
assembly.to_excel("D:\Summer 2021\교류학술제\data_cleaning\국회(전처리)(2).xlsx", index = False)
assembly

,title,count,categ,start,text
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['국가', '보안법', '폐지', '올라오', '자유', '대한민국', '월남',..."
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', '구현', '제재', '가하', '규정', '만들', '외의', '구현..."
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['사회권', '위원회', '국가', '인권', '위원회', '포괄적', '차별',..."
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연..."
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '..."
...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['한국', '심리', '학회', '상담', '심리', '학회', '임상', '심리..."
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',..."
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '휴일', '일괄적', '월요일', '정하', '의견',..."
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '역효과', '일으키', '셧다', '운제', '청소년', '보호법'..."


In [18]:
blue.text = blue_clean_words
blue.to_excel("D:\Summer 2021\교류학술제\data_cleaning\청와대(전처리)(2).xlsx", index = False)
blue

,title,count,categ,start,text
0,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,267,인권/성평등,2020-01-02,"['서울', '지방', '병무청', '검사장', '탈의실', '발견', '탈의실',..."
1,주식시장 활성화 및 소액(개미)투자자 보호,271,경제민주화,2020-01-02,"['나라', '코스', '미국', '애플', '한국', '주식', '시장', '평가..."
2,교정기관의 민낮,198,행정,2020-01-02,"['억울', '국민', '신청', '구치소', '교도관', '이주', '구치소', ..."
3,미세먼지 저감 대책,170,안전/환경,2020-01-02,"['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '일..."
4,악질세입자 방지를 위한 세입자보호법을 재정해주세요.,2127,교통/건축/국토,2020-01-02,"['아이', '부모', '식구', '아이', '성장', '나름', '단꿈', '가지..."
...,...,...,...,...,...
15310,"충북 오창 사는 33살 동생이 신경외과에서 주사 한대로 심정지, 사망하였습니다",16307,보건복지,2021-07-05,NaN
15311,오직 코로나 백신을 위해 9평 응시 신청을 한 분들의 응시 권한을 재수생과 검정고시...,387,보건복지,2021-07-05,NaN
15312,보이스피싱 피해자의 억울함을 호소합니다. 진화하고 있는 신종 보이스피싱에 대한 수사...,352,안전/환경,2021-07-05,NaN
15313,사기죄? 왜 아닙니까?,213,기타,2021-07-05,NaN
